# Modelo KRR

Este notebook aplicará o algoritmo de KRR ao dataset obtido pelo xTB.

**Autor:** Edélio Gabriel Magalhães de Jesus.

<a id='sumario'></a>
## Sumário 

- [1) Introdução](#intro)
  - [1.1) Kernel Ridge Regression (KRR)](#sobre-krr)
  - [1.2) Diferenciando do modelo SVR](#semelhancas-svr)
  - [1.3) Objetivo do notebook](#objetivo)
- [2) Desenvolvimento do modelo](#desenvolvimento-modelo)
  - [2.1) Bibliotecas necessárias](#bibliotecas)
  - [2.2) Leitura dos dados](#leitura)
  - [2.3) Split dos dados](#split)
  - [2.4) Otimização do hiperparâmetros com o optuna](#optuna)

<a id='intro'></a>
## 1) Introdução

<a id='sobre-krr'></a>
### **1.1) Kernel Ridge Regression (KRR)**

A **Kernel Ridge Regression (KRR)** combina duas ideias poderosas da aprendizagem de máquina:

1. **Regressão Ridge** (regularização L2)  
2. **Truque do Kernel** (transformações não lineares implícitas)

**`Regressão Ridge`**

A **Regressão Ridge** é um tipo de regularização, especificamente a do tipo **L2**, caracterizada por penalizar a soma dos quadrados dos pesos [1]:

$$
  L_2: \;\; \lVert \mathbf{w} \rVert_2^2 \;=\; \sum_{j=1}^m w_j^2
$$

- **Efeito:** reduz o impacto de pesos muito grandes, mas dificilmente os torna exatamente zero.  
- **Uso típico:** útil quando todas as *features* contribuem um pouco para a predição.  
- **Interpretação:** reduz a **complexidade do modelo** ao “encolher” todos os pesos, evitando que um único parâmetro domine a predição.


**`Kernel trick`**

O **Truque do Kernel** (do inglês *kernel trick*) permite projetar os dados para um **espaço de características de dimensão muito alta (ou infinita)** sem precisar calcular explicitamente essa transformação [1].

Em vez de trabalhar com as variáveis originais $\mathbf{x}$, o modelo usa uma **função kernel** $K(\mathbf{x}_i, \mathbf{x}_j)$ que mede a similaridade entre pares de amostras. Assim, a solução final depende apenas desses produtos internos no espaço transformado.

Alguns kernels comuns:

- **Linear:** $K(\mathbf{x}, \mathbf{x}') = \mathbf{x} \cdot \mathbf{x}'$
- **Polinomial:** $K(\mathbf{x}, \mathbf{x}') = (\gamma \, \mathbf{x} \cdot \mathbf{x}' + r)^d$
- **RBF (gaussiano):** $K(\mathbf{x}, \mathbf{x}') = \exp(-\gamma \| \mathbf{x} - \mathbf{x}' \|^2)$
- **Sigmoid:** $K(\mathbf{x}, \mathbf{x}') = \tanh(\gamma \, \mathbf{x} \cdot \mathbf{x}' + r)$

**`A formulação dual do KRR`**

Usando a forma dual, a solução da KRR é dada por:

$$
\hat{\mathbf{y}} = K (K + \alpha I)^{-1} \mathbf{y}
$$

onde:

- $K$ é a matriz de kernel, com $K_{ij} = K(\mathbf{x}_i, \mathbf{x}_j)$  
- $\alpha$ é o parâmetro de regularização.

Essa forma dispensa o cálculo explícito de pesos $\mathbf{w}$ e se ajusta naturalmente a qualquer kernel definido positivo.

**`Parâmetros no sckit-learn`**

O módulo *sklearn* já disponibiliza uma classe para utilizar o algoritmo *KRR* na sua API [2]. Estes são os parâmetros disponibilizados para ajuste:

- `alpha`: força da regularização L2  
- `kernel`: tipo de kernel (`'linear'`, `'rbf'`, `'poly'`, `'sigmoid'`, etc.)  
- `gamma`: controla a largura do kernel RBF (ou o peso no kernel polinomial)  
- `degree`: grau do polinômio (quando `kernel='poly'`)  
- `coef0`: termo de deslocamento em kernels polinomial ou sigmoid  

<a id='semelhancas-svr'></a>
### **1.2) Diferenciando do modelo SVR**

O algoritmo de *KRR* se assemelha ao de *SVR* pelo uso de kernels para contornar relações não lineares entre as variáveis. Existem , no entanto, algumas diferenças ´para se ressaltar:

| Aspecto | KRR | SVR |
|:--|:--|:--|
| Função de custo | Quadrática (erro médio + regularização L2) | Margem insensível ao erro (ε-insensitive loss) |
| Tipo de solução | Fechada (via inversão matricial) | Baseada em otimização convexa (programação quadrática) |
| Parâmetro principal | $\alpha$ (regularização) | $C$ (penalização) e $\epsilon$ (margem de tolerância) |
| Tempo de ajuste | Mais rápido e estável | Mais lento, especialmente em grandes datasets |
| Sensibilidade a outliers | Maior | Menor |
| Adequado para | Dados com ruído gaussiano | Casos com margens bem definidas |

![image](https://scikit-learn.org/stable/_images/sphx_glr_plot_kernel_ridge_regression_001.png)

**Fonte:** [scikit-learn](https://scikit-learn.org/stable/modules/kernel_ridge.html)


<a id='objetivo'></a>
### **1.3) Objetivo do notebook**

Diante disso, nosso objetivo será aplicar o algoritmo de *KRR*, utilizando o módulo *sklearn*, para realizar uma *task* de *regressão supervisionada* - realizando a otimização dos hiperparâmetros através do *optuna*.

[Voltar ao topo](#sumario)

<a id='desenvolvimento-modelo'></a>
## 2) Desenvolvimento do modelo

<a id='bibliotecas'></a>
### **2.1) Bibliotecas necessárias**

Antes de tudo, precisamos importar alguns módulos e funções específicas. São quatro bibliotecas principais:

1) **_pandas_**: para criação e manipulação de dataframes;  
2) **_numpy_**: para operações matemáticas e tratamento de arrays;  
3) **_plotly_**: para geração de gráficos interativos;  
4) **_sklearn_**: para implementação de modelos e ferramentas de *Machine Learning* (como divisão de dados, normalização e validação cruzada).  
5) **optuna**: para otimização dos hiperparâmetros

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import KFold, cross_val_score
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.pruners import MedianPruner
import plotly.graph_objects as go
import plotly.express as px


<a id='leitura'></a>
### **2.2) Leitura dos dados**

Vamos dar uma olhada no nosso dataset! Também visualizaremos algumas estatísticas de cada atributo, útil para as primeiras considerações.

In [18]:
SEMENTE_ALEATORIA = 367

df = pd.read_csv('./dataset_processing/xtb_dataset.csv').sample(frac=0.1, random_state=SEMENTE_ALEATORIA)

df.describe()

Dipole        E_HOMO        E_LUMO  gap_HOMO-LUMO           ZPE  \
count  12916.000000  12916.000000  12916.000000   12916.000000  12916.000000   
mean       3.202531    -10.474635     -5.757171      -4.717464      0.143381   
std        1.757499      0.593678      2.320549       2.264760      0.032783   
min        0.000000    -15.970300    -11.174700     -14.350000      0.012600   
25%        1.931000    -10.839500     -7.431775      -5.836725      0.120520   
50%        2.983000    -10.507150     -6.347100      -4.095400      0.142961   
75%        4.232000    -10.137975     -4.424275      -3.222800      0.165368   
max       26.537000     -6.421500      2.805200      -0.011700      0.268030   

                  H             U            U0             G         Delta  
count  12916.000000  12916.000000  12916.000000  12916.000000  12916.000000  
mean     -26.842614    -26.843558    -26.994638    -26.883299    -13.484293  
std        2.142806      2.142806      2.155905      2.144145      2.155905  
min      -34.330713    -34.331658    -34.452850    -34.374238    -33.305032  
25%      -28.239447    -28.240391    -28.387537    -28.280993    -14.568182  
50%      -27.090120    -27.091064    -27.271079    -27.130717    -13.207853  
75%      -25.783072    -25.784016    -25.910750    -25.822526    -12.091394  
max       -7.144001     -7.144946     -7.173899     -7.168830     -6.026081

Uma outra forma de visualizarmos as estatísticas é através do **`boxplot`**.

In [19]:
# Transformar para formato long
df_long = df.melt(var_name='Feature', value_name='Value')

# Criar boxplot 
fig = px.box(
    df_long, 
    x='Feature', 
    y='Value', 
    points='outliers',  # mostrar apenas outliers
    color='Feature',     # cores diferentes para cada feature
    title="Boxplots das Features e do Target",
    color_discrete_sequence=px.colors.qualitative.Dark2
)

# Ajustes estéticos
fig.update_layout(
    xaxis_title="Colunas",
    yaxis_title="Valores",
    boxmode='group',      # todos no mesmo eixo
    template='plotly_white',  # fundo branco limpo
    font=dict(family="Arial", size=12),
    showlegend=False
)

fig.update_traces(marker=dict(size=4))  # tamanho dos pontos outliers

fig.show()


Perceba que as escalas das *features* não são muito diferentes - com destaque para o *ZPE*, que possui valores muito pequenos em torno de 0 -, mas existem alguns *outliers*. 

O algoritmo *KRR* é bastante sensível à escala e valores extremos, justamente devido ao uso de *kernls* - como o *rbf*. Portanto, é importanto que a normalização dos dados faça parte do nosso *pipeline*.

---

`Observação:` O gráfico foi plotado com *plotly*, que permite ampliar os gráfico - basta selecionar a área desejada.

<a id='split'></a>
### **2.3) Split dos dados**

A etapa de *split* no tratamento de dados consiste em `dividir` o conjunto de dados disponível em subconjuntos distintos: *`treinamento`* e *`teste`*. Essa divisão é importante para que o modelo de aprendizado de máquina seja treinado em uma parte dos dados e, posteriormente, avaliado em dados que ele nunca viu, garantindo uma medição justa de seu desempenho e evitando o *overfitting* - o ajuste excessivo aos dados de treinamento - e o *underfitting* - subestimação dos dados.

In [20]:
TAMANHO_TESTE = 0.2

X = df.drop(columns=['Delta'])
y = df['Delta']

X_treino, X_teste, y_treino, y_teste =train_test_split(X, y, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA)

Meste notebook, estaremos utilizando 80% dos dados para realizar o treino do modelo - via optuna - e 20% para a previsão - quando veremos o real desempenho do modelo com dados não vistos.

<a id='optuna'></a>
### **2.4) Otimização dos hiperparâmetros com o optuna**

#### **`Modelos e espaço de busca`**

A função `cria_instancia_modelo` abaixo serve para criar uma instância do modelo escolhido. Esta função recebe um objeto tipo *trial*; do `optuna`.

Observe que o dicionário `parametros` dentro desta função tem como chaves os nomes dos argumentos do modelo. Os valores dos argumentos, por sua vez, podem ser sorteados com as funções

-   `trial.suggest_int` (para números inteiros)
-   `trial.suggest_float` (para números reais) e
-   `trial.suggest_categorical` (para dados categóricos).

São com estas funções que delimitamos o **espaço de busca** dos hiperparâmetros do modelo.

Além dos parâmetros do modelo, estamos permitindo que o *optuna* decida se irá realizar os pré-processamentos de reduzir a dimensionalidade (via *PCA*).

In [21]:
def criar_instancia_modelo_krr(trial):
    # Escolha do kernel
    kernel_escolhido = trial.suggest_categorical(
        "kernel", ["rbf", "poly", "linear", "laplacian"]
    )

    # Gamma contínuo para kernels baseados em distância
    if kernel_escolhido in ["rbf", "poly", "laplacian"]:
        gamma = trial.suggest_float("gamma", 1e-4, 10, log=True)
    else:
        gamma = 'scale'

    # Alpha (regularização)
    alpha = trial.suggest_float("alpha", 1e-5, 10, log=True)

    # Grau do polinomial (somente se kernel == "poly")
    degree = trial.suggest_int("degree", 2, 7) if kernel_escolhido == "poly" else 3

    parametros_krr = {
        "alpha": alpha,
        "kernel": kernel_escolhido,
        "gamma": gamma,
        "degree": degree,
        "coef0": trial.suggest_float("coef0", 0.0, 10.0),
    }

    # Pipeline
    steps = [("normalizador", StandardScaler())]

    # Decisão de aplicar PCA (apenas para kernels baseados em distância)
    usar_pca = trial.suggest_categorical("usar_pca", [True, False])
    if usar_pca and kernel_escolhido in ["rbf", "poly", "laplacian"]:
        n_comp = trial.suggest_float("variancia_pca", 0.85, 0.99)
        steps.append(("pca", PCA(n_components=n_comp, random_state=SEMENTE_ALEATORIA)))

    # Regressor KernelRidge
    steps.append(("regressor", KernelRidge(**parametros_krr)))

    modelo = Pipeline(steps=steps)

    return modelo

#### `Função objetivo`

A **função objetivo** em um problema de otimização é responsável por calcular a **métrica de desempenho** que será minimizada ou maximizada durante o processo.  

Neste caso, a métrica escolhida é o **RMSE (Root Mean Squared Error)**, obtido por meio de **validação cruzada**.

---

**Observação:**  
O *Python* (ou mais especificamente, algumas funções do `scikit-learn`) retorna valores **negativos** para métricas de erro quando configuradas para maximização.  
Por isso, é necessário **multiplicar por -1** antes de calcular a média dos valores de RMSE, garantindo que o algoritmo do Optuna minimize corretamente o erro.


In [22]:
from sklearn.model_selection import KFold, cross_val_score

def funcao_objetivo(trial, X, y, NUM_FOLDS=10):
    # Cria o modelo SVR usando o pipeline
    modelo = criar_instancia_modelo_krr(trial)

    # KFold para regressão
    cv = KFold(
        n_splits=NUM_FOLDS,
        shuffle=True,
        random_state=SEMENTE_ALEATORIA
    )

    # Cross-validation usando RMSE como métrica
    metricas = cross_val_score(
        modelo,
        X,
        y,
        scoring="neg_root_mean_squared_error",  # para regressão
        cv=cv,
        n_jobs=-1
    )

    # Retorna a média do RMSE
    return -metricas.mean()


#### `Otimização`

A otimização dos hiperparâmetros é conduzida por meio da criação de um **objeto de estudo** com a função `create_study`.

No nosso caso, como a métrica de interesse é o **RMSE (Root Mean Squared Error)** — um indicador de erro —, quanto **menor** o seu valor, **melhor** o desempenho do modelo.  
Por isso, atribuímos o valor `"minimize"` ao argumento `direction`.

O parâmetro `study_name` define um **nome identificador** para o processo de busca, permitindo que possamos retomá-lo posteriormente.  
Já o argumento `storage` especifica o **local onde os resultados do estudo serão armazenados**, geralmente em um arquivo de banco de dados no formato SQLite (`.db`).

Por fim, o argumento `load_if_exists=True` instrui o Optuna a **verificar se já existe um estudo com o mesmo nome**.  
Caso exista, o novo processo será **incorporado ao estudo anterior**, permitindo continuar a otimização de forma cumulativa.


In [ ]:
NOME_DO_ESTUDO = "svr_xtb_dataset_krr_1"
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

# Criar ou carregar estudo existente
objeto_de_estudo = create_study(
    direction="minimize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
    sampler=TPESampler(seed=SEMENTE_ALEATORIA)
)

# Forçar algumas combinações iniciais
objeto_de_estudo.enqueue_trial({"usar_pca": False})
objeto_de_estudo.enqueue_trial({"usar_pca": True})

# Função-objetivo parcial
def funcao_objetivo_parcial(trial):
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)

# Otimização
objeto_de_estudo.optimize(funcao_objetivo_parcial, n_trials=NUM_TENTATIVAS)

[I 2025-10-27 10:16:42,935] Using an existing study with name 'svr_xtb_dataset_krr_1' instead of creating a new one.
[W 2025-10-27 10:17:06,734] Trial 1 failed with parameters: {'kernel': 'laplacian', 'gamma': 0.00020938532907692294, 'alpha': 0.0001624794755205252, 'coef0': 9.102443309867745, 'usar_pca': True, 'variancia_pca': 0.9846109686274004} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\edelio25024\OneDrive - ILUM ESCOLA DE CIÊNCIA\VSCODE\MACHINE_LEARNING\.venv\Lib\site-packages\optuna\study\_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\edelio25024\AppData\Local\Temp\ipykernel_1232\246422125.py", line 20, in funcao_objetivo_parcial
    return funcao_objetivo(trial, X_treino, y_treino, NUM_FOLDS)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\edelio25024\AppData\Local\Temp\ipykernel_1232\3067305631.py", line 15, 

KeyboardInterrupt: 